# 1. 在MINIST-FASHION上实现神经网络的学习流程

本节课我们讲解了神经网络使用小批量随机梯度下降进行迭代的流程，现在我们要整合本节课中所有的代码实现一个完整的训练流程，首先要梳理一下整个流程：

1. 设置步长 $lr$,动量值 $gamma$，迭代次数 $epochs$，$batchsize$等信息，（如果需要）设置初始权重 $ w_{0}$
2. 导入数据集，将数据切分成batches
3. 定义神经网络架构
4. 定义损失函数L（w），如果需要的话，将损失函数调成凸函数，以便求解最小值
5. 定义所使用的优化算法
6. 开始在epoches和batch上循环，执行优化算法：
    a. 调整数据结构，确定数据能够在神经网络，损失函数和优化算法中顺利运行
    b.完成向前传播，在损失函数L（w）上对每一个w求偏导数
    c.迭代当前权重
    d.清空本轮梯度
    

In [ ]:
import torchvision
import torchvision.transforms as transforms	    # 数据处理模块

# dataloader, tensordataset - 对数据结构、归纳方式进行变换
# torchvision.transforms - 对数据集的数字本身进行修改

mnist = torchvision.datasets.FashionMNIST(root="./data"   # 你的计算机上的某个目录
                                        , download= True
                                        , train= True
                                        , transform= transforms.ToTensor()
                                        )         # 实例化数据

In [ ]:
# 导入库、包
import torch
import torch.nn as nn 
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss as CEL 

# 设置超参数
lr = 0.01
gamma = 0.5
epochs = 5
batch_size = 1000

# 导入数据，并将数据切分成batches
mnist = torchvision.datasets.FashionMNIST(root= './data'
                                          , download= True
                                          , train= True
                                          , transform= transforms.ToTensor()
                                          )
mnist_dataset = DataLoader(mnist
                           ,batch_size= batch_size
                           , shuffle= True
                            , drop_last= True
                            )

in_put = mnist.data.shape[0]                # in_put输入有问题，输入层应该是28*28
out_put = mnist.targets.unique()            # out_put输出也有问题，应该用len()函数

# 神经网络架构
class Model(nn.Module):
    def __init__(self, in_features=10, out_features=2):
        super().__init__()
        self.Linear_1 = nn.Linear(in_features, 13)
        self.Linear_2 = nn.Linear(13, 8)
        self.out = nn.Linear(8, out_features)
    
    def forward(self, X):
        z1 = self.Linear_1(X)           # 要考虑考虑X是否需要进行形状的变换，因为只能传入一维的数据
        sigma_1 = F.sigmoid(z1)
        z2 = self.Linear_2(sigma_1)
        sigma_2 = F.relu(z2)
        zhat = self.out(sigma_2)
        return zhat

# 损失函数，反向传播，优化算法可以考虑放置在同一个函数里面

model = Model(in_put, out_put)
# 定义损失函数L(w)
def Loss(zhat, y):
    criterion = CEL()       # 如果要输入模型的准确率，就不能用CEL损失函数进行计算
    loss = criterion(zhat, y)
    return loss
optimizer = optim.SGD(model.parameters(), lr= lr)
for epochs in range(epochs):
    for batch_idx, (data, target) in enumerate(mnist_dataset):
        # 梯度清零
        optimizer.zero_grad()
        # 向前传播
        zhat = model.forward(data)
        # 计算损失函数
        loss = Loss(zhat, target)
        # 计算梯度
        loss.backward()
        optimizer.step

        




准确率仅仅为47%，这是由于神经网络的结构，以及激活函数导致的：

Sigmoid激活函数在图像识别分类任务中不太常用，而更常见的选择是Rectified Linear Unit (ReLU) 激活函数及其变种，如Leaky ReLU、Parametric ReLU等。这是因为Sigmoid函数在某些情况下存在梯度消失的问题，导致深层神经网络的训练变得困难，尤其是在反向传播过程中，梯度逐渐减小，可能导致网络权重更新变得缓慢，从而降低了模型的训练速度和性能。

相比之下，ReLU函数具有以下优点：

* 避免梯度消失问题： ReLU函数在正数区间的梯度恒为1，这有助于防止梯度消失问题，使得深层网络的训练更加稳定和高效。

* 计算高效： ReLU函数的计算相对简单，只需要比较输入是否大于零即可，相比于Sigmoid函数和双曲正切函数的计算更快。

* 稀疏激活性： ReLU的负值部分会变为零，从而使得神经元具有一定的稀疏激活性，这有助于模型的泛化能力。

虽然Sigmoid函数在输出范围上有界（0到1之间），适用于二分类问题，但在深度神经网络中，ReLU激活函数通常更适合处理图像识别分类任务。如果你关心输出范围，你可以使用输出范围在[0, 1]的输出层来处理分类问题，例如使用Sigmoid激活函数，但是在隐藏层中，使用ReLU激活函数会更为常见和推荐。

In [ ]:
# 修改后的代码

# 导入库、包
import torch
import torch.nn as nn 
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss as CEL 

# 设置超参数
lr = 0.01
gamma = 0.5
epochs = 5
batch_size = 124

# 导入数据，并将数据切分成batches
mnist = torchvision.datasets.FashionMNIST(root= './data'
                                          , download= True
                                          , train= True
                                          , transform= transforms.ToTensor()
                                          )
batch_data = DataLoader(mnist
                           ,batch_size= batch_size
                           , shuffle= True
                            , drop_last= True
                            )

in_put = mnist.data[0].numel()
out_put = len(mnist.targets.unique())

# 神经网络架构
class Model(nn.Module):
    def __init__(self, in_features=10, out_features=2):
        super().__init__()
        self.Linear_1 = nn.Linear(in_features, 13)
        self.Linear_2 = nn.Linear(13, 8)
        self.out = nn.Linear(8, out_features)
    
    def forward(self, X):
        z1 = self.Linear_1(X.view(-1, 28*28))
        sigma_1 = F.sigmoid(z1)
        z2 = self.Linear_2(sigma_1)
        sigma_2 = F.relu(z2)
        zhat = self.out(sigma_2)
        sigma = F.log_softmax(zhat, dim = 1)
        return sigma

def fit_(net, batch_data,lr= 0.05, gamma= 0.5,epochs= 5):
  criterion = nn.NLLLoss()
  opt = optim.SGD(net.parameters()
            ,lr = lr
            , momentum = gamma)
  samples = 0 # 循环开始之前，模型一个样本都没见过
  correct = 0 # 循环开始之前，训练样本为0
  for epoch in range(epochs):
    for batch_idx, (x,y) in enumerate(batch_data):
      y = y.view(x.shape[0])
      sigma = net.forward(x)
      loss = criterion(sigma, y)
      loss.backward()
      opt.step()
      opt.zero_grad()

      # 求解准确率，全部判断正确的样本数据/已经看过的总样本量
      yhat = torch.max(sigma, 1)[1]
      correct += torch.sum(yhat == y).item()
      samples += x.shape[0]

      if (batch_idx +1) % 125 == 0:
        print("Epoch{}:[{}/{} {:.0f}%, Loss:{:.6f},Accuracy:{:.3f}]".format(epoch+1
                      , samples
                      , epochs*len(batch_data.dataset)
                      , 100*samples/(epochs*len(batch_data.dataset))
                      , loss.data.item()
                      , float(100*correct/samples)))

# 训练与评估
torch.manual_seed(1412)
net = Model(in_features = in_put, out_features= out_put)
fit_(net, batch_data, lr= lr, gamma= gamma, epochs = epochs)




菜菜的代码：

In [ ]:

# 导入库
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms

# 确定数据、超参数
lr = 0.15
gamma = 0.8
epochs = 5
bs = 128

mnist = torchvision.datasets.FashionMNIST(root= './data'
                                          , download= True
                                          , train= True
                                          , transform= transforms.ToTensor()
                                          )
batch_data = DataLoader(mnist
                        , batch_size= bs
                        , shuffle= True
                        , drop_last= True)

in_put = mnist.data[0].numel()
out_put = len(mnist.targets.unique())

class Model(nn.Module):
  def __init__(self, in_features = 10, out_features = 2):
      super().__init__()
      self.linear1 = nn.Linear(in_features, 128, bias = False)
      self.out = nn.Linear(128, out_features, bias = False)
  
  def forward(self, x):
    X = x.view(-1, 28*28)   # view(-1),表示需要对数据结构进行一个改变，-1作为占位符，表示请pytorch帮助我们自动计算-1处的维度
    sigma1 = torch.relu(self.linear1(X))
    sigma2 = F.log_softmax(self.out(sigma1), dim = 1)
    return sigma2

def fit_(net, batch_data, lr=0.01, epochs=5, gamma= 1.0):
  criterion = nn.NLLLoss()
  opt = optim.SGD(net.parameters()
            , lr = lr
            , momentum= gamma)
  samples = 0 # 循环开始之前，模型一个样本没见过
  correct = 0 # 循环开始之前，训练样本为0
  for epoch in range(epochs):
    for batch_idx, (x,y) in enumerate(batch_data):
      # y = y.view(x.shape[0]) 降维
      sigma = net.forward(x)
      loss = criterion(sigma, y)
      loss.backward()
      opt.step()
      opt.zero_grad()

      # 求解准确率，全部判断正确的样本数据/已经看过的总样本量
      yhat = torch.max(sigma, 1)[1]   # torch.max函数结果中的索引为1的部分
      correct += torch.sum(yhat == y).item()

      samples += x.shape[0]
      if (batch_idx + 1) % 125 == 0:
        print("Epoch{}:[{}/{} {:.0f}%, Loss:{:.6f},Accuracy:{:.3f}]".format(epoch+1
                      , samples
                      , epochs*len(batch_data.dataset)
                      , 100*samples/(epochs*len(batch_data.dataset))
                      , loss.data.item()
                      , float(100*correct/samples)))  # 分子代表：已经查看过的数据有多少，分母代表在现有的epochs设置，模型一共需要查看多少数据
        
# 训练与评估
torch.manual_seed(1412)
net = Model(in_features= in_put, out_features= out_put)
fit_(net, batch_data, lr =lr, epochs= epochs, gamma= gamma)

注释：
1. torch.max(): 结果是一个包含两个张量的元组，第一个包含了最大值，第二个包含了最大值的索引


我们现在已经完成了一个最基本的、神经网络训练并查看训练结果的代码，是不是感觉已经获得了很多知识呢？我们的模型最后得到的结果属于中规中矩，毕竟我们设置的网络结构只是最普通的全连接层，并且我们没有对数据进行任何处理或增强。已经成熟的神经网络架构可以很轻易在MINST-FASHION数据集上获得99%的准确率，因此我们还有很长的路要走。